# Codescan w/ Codesplitter

## Setup

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
import mlflow

mlflow.llama_index.autolog()

mlflow.set_tracking_uri("http://localhost:5000")

## Models

In [4]:
LLM = "deepseek-r1"
EMBED_MODEL = "jinaai/jina-embeddings-v2-base-code"

In [5]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


Settings.llm = Ollama(
    LLM,
    request_timeout=3 * 60,
    async_mode=True,
)
Settings.embed_model = HuggingFaceEmbedding(EMBED_MODEL)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.44k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/322M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/971k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

## Reading data

In [ ]:
from llama_index.readers.github import GithubClient, GithubRepositoryReader


github_client = GithubClient(github_token="TOKEN")

# Any repo works, though it was tested on Java code explicitly
OWNER = "cruderra"
REPO = "food-delivery-demo"
BRANCH = "main"

reader = GithubRepositoryReader(
    github_client=github_client,
    owner=OWNER,
    repo=REPO,
    filter_file_extensions=(
        ".java",
        GithubRepositoryReader.FilterType.INCLUDE,
    ),
)

documents = reader.load_data(branch=BRANCH)
documents = list(
    filter(
        lambda d: d.metadata["file_name"].endswith(".java"),
        documents,
    )
)
documents

[Document(id_='2d5e281f435fe2d9e7d34cdb7a35b8c699b858d9', embedding=None, metadata={'file_path': 'api-gateway/src/main/java/api/gateway/com/ApiGatewayApplication.java', 'file_name': 'ApiGatewayApplication.java', 'url': 'https://api.github.com/cruderra/food-delivery-demo/blob/main/api-gateway/src/main/java/api/gateway/com/ApiGatewayApplication.java'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='package api.gateway.com;\n\nimport org.springframework.boot.SpringApplication;\nimport org.springframework.boot.autoconfigure.SpringBootApplication;\n\n@SpringBootApplication\npublic class ApiGatewayApplication {\n\n\tpublic static void main(String[] args) {\n\t\tSpringApplication.run(ApiGatewayApplication.class, args);\n\t}\n\n}\n', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_t

## Creating the graph

In [6]:
mlflow.set_experiment("codescan w/ codesplitter")

<Experiment: artifact_location='mlflow-artifacts:/319831797427190372', creation_time=1738855480692, experiment_id='319831797427190372', last_update_time=1738855480692, lifecycle_stage='active', name='codescan w/ codesplitter', tags={}>

In [7]:
from llama_index.core import PropertyGraphIndex
from llama_index.core.node_parser import CodeSplitter
# from llama_index.core.indices.property_graph import SchemaLLMPathExtractor


with mlflow.start_run():
    index = PropertyGraphIndex.from_documents(
        documents,
        transformations=[CodeSplitter("java")],
        use_async=True,
        # show_progress=True,
    )

/home/sv-cheats-1/Documents/Work/Cruderra/ml-experiments/.venv/lib/python3.11/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)
BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


🏃 View run persistent-pig-764 at: http://localhost:5000/#/experiments/319831797427190372/runs/3b9ab24108da44288776cfdb6cf8e30b
🧪 View experiment at: http://localhost:5000/#/experiments/319831797427190372


[Trace(request_id=97f0207c62f1491da55fd316b6feb0e2), Trace(request_id=bc0034f285f44a59817f2c4d7328366d), Trace(request_id=eae748f79164444588922ee2d0737589), Trace(request_id=e0331b0a37644aa0a9ef53c3b22c2345), Trace(request_id=5adebf98fd0b485caa8f3c8635e11154)]

In [8]:
index.property_graph_store.show_jupyter_graph()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [9]:
index.storage_context.persist('./graph')

## Testing

### Load index from storage

In [7]:
from llama_index.core import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir="./graph")
index = load_index_from_storage(storage_context)

In [9]:
from llama_index.core.chat_engine.types import ChatMode

chat_engine = index.as_chat_engine(chat_mode=ChatMode.CONTEXT)

In [10]:
response = chat_engine.chat("Write a technical design document for the code. Do not go into too much detail, just the high-level overview. Define flows, what modules exist and how they interact with eachother. ")

2025/02/17 15:06:31 WARNING mlflow.tracing.processor.mlflow: Creating a trace within the default experiment with id '0'. It is strongly recommended to not use the default experiment to log traces due to ambiguous search results and probable performance issues over time due to directory table listing performance degradation with high volumes of directories within a specific path. To avoid performance and disambiguation issues, set the experiment for your environment using `mlflow.set_experiment()` API.


2025/02/17 15:10:37 WARNING mlflow.tracking.client: Failed to start trace ContextChatEngine.chat: API request to http://localhost:5000/api/2.0/mlflow/traces failed with exception HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/2.0/mlflow/traces (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7a4f144eb850>: Failed to establish a new connection: [Errno 111] Connection refused')). For full traceback, set logging level to debug.
BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [16]:
print(response.response)

<think>
Alright, so I need to write a technical design document based on the provided code snippets. Let me start by understanding what's given.

First, there are several Java files from different packages: OrderApplicationTests.java, OrderService.java, DeliveryController.java, DeliveryService.java, and FoodService.java. The context mentions that these are part of an order management system with API endpoints for food delivery.

Looking at the test file, it's a JUnit test case in the OrderApplicationTests class. It has a @Test method called contextLoads() which seems to be where setup happens for testing contexts, probably using Spring Boot annotations like @BeforeEach and @AfterEachContext.

Next, OrderService.java shows that it's an @Service with @RequiredArgsConstructor, meaning it uses Lombok for constructor injection. The service creates orders by making POST requests to a URL stored in the @Value annotation.

Then there's DeliveryController.java, which uses Spring annotations lik

In [17]:
from IPython.display import display, Markdown, Latex
display(Markdown(response.response))

<think>
Alright, so I need to write a technical design document based on the provided code snippets. Let me start by understanding what's given.

First, there are several Java files from different packages: OrderApplicationTests.java, OrderService.java, DeliveryController.java, DeliveryService.java, and FoodService.java. The context mentions that these are part of an order management system with API endpoints for food delivery.

Looking at the test file, it's a JUnit test case in the OrderApplicationTests class. It has a @Test method called contextLoads() which seems to be where setup happens for testing contexts, probably using Spring Boot annotations like @BeforeEach and @AfterEachContext.

Next, OrderService.java shows that it's an @Service with @RequiredArgsConstructor, meaning it uses Lombok for constructor injection. The service creates orders by making POST requests to a URL stored in the @Value annotation.

Then there's DeliveryController.java, which uses Spring annotations like @RestController and @PostMapping. It injects DeliveryService as a binding and maps the create method to an endpoint "/delivery". The controller takes a String body parameter and passes it to the service.

Moving on to DeliveryService.java, this class has a create method that accepts a String body without any parameters beyond what's injected. It seems straightforward but might be called by the DeliveryController.

Lastly, FoodService.java uses RestTemplate annotations like @RestTemplate to handle creating orders via POST requests to both order and delivery URLs. The service is also annotated with @RequiredArgsConstructor for Lombok injection.

Putting it all together:

1. **Order Management Layer**: This includes OrderService which handles creating orders by making HTTP POST requests to a specific endpoint. It's probably responsible for the core business logic of processing orders.

2. **Delivery Controller**: This layer interacts with DeliveryService, which is likely where the actual delivery data and operations are handled. The controller uses Spring annotations to map its create method to an API endpoint.

3. **Food Gateway Service**: Using RestTemplate, this service handles creating orders by making POST requests to both order and delivery endpoints. It's part of the food gateway layer, integrating with other systems like OrderService and DeliveryService.

**Interactions**:
- The Food Gateway Service interacts with OrderService and DeliveryService by calling their create methods after processing the incoming request.
- The DeliveryController injects DeliveryService and uses it to handle order creation specifically for deliveries.
- The OrderService is used both directly (by FoodService) and through its injected bindings.

Potential issues or considerations might include handling different types of orders, ensuring data consistency between layers, and managing API endpoints correctly. Also, testing would be crucial to ensure that each service layer works as expected when called from different contexts like the food gateway versus direct order processing.
</think>

**Technical Design Document**

**1. Overview**
This system is designed for a food delivery platform with three main layers: Order Management, Delivery Controller, and Food Gateway Service.

**2. Modules**

- **Order Management Layer (OrderService.java)**
  - Implements core order creation functionality by making HTTP POST requests.
  - Uses Lombok's @RequiredArgsConstructor for constructor injection to handle parameters like payment URLs.
  - Stores orders via REST API calls mapped to endpoints such as /delivery.

- **Delivery Controller (DeliveryController.java)**
  - Acts as the entry point for delivering food through RESTful web services.
  - Injects DeliveryService for order creation and maps its create method to an endpoint (/delivery).
  - Consumes JSON request bodies formatted with delivery details like driver name, route, etc.

- **Food Gateway Service (FoodService.java)**
  - Integrates with external systems via RESTTemplate annotations.
  - Makes API calls to both OrderService and DeliveryService endpoints when processing orders from the food gateway layer.
  - Handles creation of orders by invoking these services with request bodies containing necessary data like order ID, user ID, etc.

**3. End-to-End Flow**

1. **Order Creation via Food Gateway Service**
   - Receives a JSON body containing order details (user_id, order_id).
   - Uses RestTemplate to make POST requests to both OrderService and DeliveryService endpoints with the provided request body.
   - Exports a food delivery ticket by combining the received JSON data with additional information from the services.

2. **Order Creation via Direct Route**
   - Order Service receives a request for creating an order, typically through its REST API endpoint (/delivery).
   - Makes POST requests to both OrderService and DeliveryService endpoints using RestTemplate.
   - Returns an order ticket containing structured data like order ID, user ID, etc.

**4. Interactions Between Modules**

- **Order Management Layer (OrderService)**
  - Used directly by Food Gateway Service via RESTTemplate annotations for order creation.
  - Also injected into other services where needed.

- **Delivery Controller**
  - Injects DeliveryService and maps its create method to an endpoint.
  - Consumes JSON request bodies formatted with specific delivery details, passed as parameters in request body format.

**5. Challenges**

- Ensuring data consistency between layers
- Correct handling of different types of orders (e.g., food delivery vs direct)
- Efficient management of API endpoints for order and delivery services